https://developer.github.com/v3/repos/statistics/

In [150]:
    %%capture
    import uritemplate, requests, pandas, hvplot.pandas; __import__('requests_cache').install_cache('stats')

In [151]:
    stats = "https://api.github.com/repos/{owner}/{repo}/stats/"

In [152]:
    for x in "contributors commit_activity code_frequency".split():
        globals()[x] = uritemplate.URITemplate(stats+x)

In [153]:
    owner, repo = "omnisci/jupyterlab-omnisci".split('/')

In [154]:
    def get(url):
        if requests.get(url).status_code == 202: return __import__('time').sleep(1) or get(url)
        return pandas.DataFrame(requests.get(url).json())

In [155]:
    df = get(contributors.expand(globals()))
    df = df.weeks.apply(pandas.Series).stack().apply(pandas.Series).reset_index(-1, drop=True).join(df.author.apply(pandas.Series))
    df.w = df.w.pipe(pandas.to_datetime, unit='s')
    df = df.set_index(['w', 'login'])

    contributions = df[list('acd')].rename(columns=dict(zip('ad', '+-'), c='commits')).unstack('login')
    contributions.T

w                       2018-03-25  2018-04-01  2018-04-08  2018-04-15  \
        login                                                            
+       domoritz                 0           0           0           0   
        gnestor                  0           0           0           0   
        ian-r-rose           42848         507         771         319   
        saulshanabrook           0           0           0           0   
        tonyfast                 0           0           0           0   
commits domoritz                 0           0           0           0   
        gnestor                  0           0           0           0   
        ian-r-rose              14           3          10           5   
        saulshanabrook           0           0           0           0   
        tonyfast                 0           0           0           0   
-       domoritz                 0           0           0           0   
        gnestor                  0           0           0           0   
        ian-r-rose           42164          17         353          78   
        saulshanabrook           0           0           0           0   
        tonyfast                 0           0           0           0   

w                       2018-04-22  2018-04-29  2018-05-06  2018-05-13  \
        login                                                            
+       domoritz                 0           0           0           0   
        gnestor                  0           0           0           0   
        ian-r-rose             551         311           0           0   
        saulshanabrook         410           0          70         456   
        tonyfast                15           0           0           0   
commits domoritz                 0           0           0           0   
        gnestor                  0           0           0           0   
        ian-r-rose               8           9           0           0   
        saulshanabrook           8           0           1           3   
        tonyfast                 1           0           0           0   
-       domoritz                 0           0           0           0   
        gnestor                  0           0           0           0   
        ian-r-rose             111         103           0           0   
        saulshanabrook          56           0          33         127   
        tonyfast                 1           0           0           0   

w                       2018-05-20  2018-05-27  ...  2019-08-11  2019-08-18  \
        login                                   ...                           
+       domoritz                 0           0  ...           0           0   
        gnestor                  0           0  ...           0           0   
        ian-r-rose               0           0  ...           0           0   
        saulshanabrook          28         225  ...        1671           0   
        tonyfast                 0           0  ...           0           0   
commits domoritz                 0           0  ...           0           0   
        gnestor                  0           0  ...           0           0   
        ian-r-rose               0           0  ...           0           0   
        saulshanabrook           2           7  ...          10           0   
        tonyfast                 0           0  ...           0           0   
-       domoritz                 0           0  ...           0           0   
        gnestor                  0           0  ...           0           0   
        ian-r-rose               0           0  ...           0           0   
        saulshanabrook          28          12  ...        4720           0   
        tonyfast                 0           0  ...           0           0   

w                       2019-08-25  2019-09-01  2019-09-08  2019-09-15  \
        login                                                   

In [156]:
    df = get(commit_activity.expand(globals()))
    df.week = df.week.pipe(pandas.to_datetime, unit='s')
    df = df.set_index('week').days.apply(pandas.Series)
    df.columns = (
        df.index[0]+pandas.to_timedelta(df.columns.map('{} days'.format))
    ).strftime('%A')

    weekly_commits = df
    weekly_commits.T

week,2018-10-21,2018-10-28,2018-11-04,2018-11-11,2018-11-18,2018-11-25,2018-12-02,2018-12-09,2018-12-16,2018-12-23,...,2019-08-11,2019-08-18,2019-08-25,2019-09-01,2019-09-08,2019-09-15,2019-09-22,2019-09-29,2019-10-06,2019-10-13
Sunday,1,0,0,8,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Monday,0,8,9,3,3,3,1,0,13,0,...,9,0,0,0,0,0,0,0,0,0
Tuesday,0,11,0,4,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
Wednesday,0,0,0,0,0,0,0,2,0,1,...,0,0,0,0,0,0,0,0,0,0
Thursday,0,0,0,0,0,0,0,0,0,7,...,0,0,2,0,0,0,0,0,0,0
Friday,4,1,0,0,0,3,0,0,0,0,...,0,0,3,4,0,0,0,0,0,0
Saturday,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [157]:
    df = get(code_frequency.expand(globals()))
    df = df[[1, 2]].rename(columns=dict(zip([1,2], "+-"))).set_index(df[0].pipe(pandas.to_datetime, unit='s').rename('week'))
    code_changes = df; code_changes.T

week,2018-03-25,2018-04-01,2018-04-08,2018-04-15,2018-04-22,2018-04-29,2018-05-06,2018-05-13,2018-05-20,2018-05-27,...,2019-08-11,2019-08-18,2019-08-25,2019-09-01,2019-09-08,2019-09-15,2019-09-22,2019-09-29,2019-10-06,2019-10-13
+,42848,507,771,319,976,311,70,456,28,225,...,1671,0,656,149,0,0,0,0,0,0
-,-42164,-17,-353,-78,-168,-103,-33,-127,-28,-12,...,-4720,0,-1814,-73,0,0,0,0,0,0
